# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [2]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 300)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

In [4]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

# Create text_dict

In [5]:
import os

folder_path = '../text/2023-04-18'
encoding='ISO-8859-1'
all_files = []

for filename in os.listdir(folder_path):
    with open(os.path.join(folder_path, filename), 'r', encoding=encoding) as f:
        all_files.append(f.read())

text_dict = create_text_dict(all_files)
text_dict

{1: 'Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, an internal focus elicited superior hypertrophic increases in the elbow flexors compared to an external focus, but MT in the quadriceps was unaffected by attentional focus strategy. The differences in changes in elbow flexor size between INTERNAL and EXTERNAL (12.4% vs. 6.9%, respectively) translated into a large magnitude of effect favouring the INTERNAL condition (h2 p = 0.307). These findings partially support the common bodybuilding claim that a mind\x96muscle connection enhances muscle growth. Although we did not attempt to determine mechanistic reasons for discrepant findings between the upper and lower limbs, it can be speculated that subjects found it easier to focus on the elbow fl

## Create prompts_df

In [6]:
from itertools import product
prompts_dict = dict()
experiment_num = 1
relevance_prompts_dict = dict()

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]
task_part2 = [
    # "Use language for a lay audience",
    "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

# prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
#     columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

summarize_task = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])

relevance_prompts_df = pd.DataFrame(product(task_part3, audience), 
    columns=['task part 3', 'audience'])



# prompts_df['prompt'] = prompts_df.apply(
#     lambda row: f"{row['prep_step']} {row['task part 1']}.", 
#     axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
relevance_prompts_df['relevance'] = relevance_prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
relevance_prompts_dict[experiment_num] = relevance_prompts_df
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


# Iteration 1

In [8]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.summaries_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.summaries_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        print('relevance_iteration: ', relevance_iteration)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance statement': choice["message"]["content"],
                        'relevance task': relevance_task,
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index, prompt in enumerate(simplify_prompts):
            # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            # print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='simple'):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "Added relevance"} dataframe shape:', new_results.shape)
    
    return new_results


iteration_id = 1
n_choices = 2
qna_dict = dict()
simple_summaries_dict = dict()
chatbot_dict = dict()
relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
relevance_prompts_df

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    # chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# simple_summaries_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id)

# relevance_dict = prompt_chaining_dict(relevance_prompts_df, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )

# relevance_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='relevance')

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

In [9]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,prep step,summarization task,summary,text
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertr...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that different attentional focus strategies have varying effects on long-term muscular adaptations during resistance training. An internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but attentional focus strategy di...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read a really interesting study on the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but the same focus did ...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
3,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the latest study on attentional focus during resistance training? It found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, which partially supports the bodybuilding claim that a mind-mus...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
4,1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that weight stigma is associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study, which used a large sample of U.S. adults, highlights weight stigma as a potential barrier to healthy behaviors

In [10]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )
simple_summaries_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id)


summary_iteration_id: 1
n_choices: 1
**text1_prompt00
simplify_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
simplify_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending 

In [19]:
chaining_dict[iteration_id]['text1_prompt00'].relevance_dict

{5: {'response_01': {0: {'relevance choice': 1,
    'relevance statement': 'For athletes who want to optimize their muscle growth, these findings suggest that focusing on internal cues, such as the feeling of the muscle contracting, may be more effective for certain muscle groups. Incorporating this attentional focus strategy into their resistance training routine could potentially lead to greater gains in muscle size and strength.',
    'relevance task': ' Add 1-2 sentences to make this relevant for people who enjoy sports ',
    'preceding summary': 'A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertrophic outcomes and may be muscle-specific. These findings provide insights into maximizing muscle growth and the mind-muscle connection in resist

In [13]:
simple_summaries_dict

,choice,article_title,system_role,model,prep step,summarization task,summary,text,original summary,simple summary,simple summary choice,simplify task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertr...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertr...",A study looked at how people can make their muscles bigger by lifting weights for a long time. They found that thinking about the muscles you are using while lifting weights can make some muscles grow better than others. This means that you can use your mind to help your muscles get bigger when ...,1,Use terms a 12-year-old can understand.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that different attentional focus strategies have varying effects on long-term muscular adaptations during resistance training. An internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but attentional focus strategy di...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A new study has found that different attentional focus strategies have varying effects on long-term muscular adaptations during resistance training. An internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but attentional focus strategy di...","A new study found that when people lift weights, thinking about their muscles growing bigger (internal focus) helps their arm muscles get bigger more than thinking about the weight they are lifting (external focus). But, it doesn't make a difference for leg muscles. This study shows that how you...",1,Use terms a 12-year-old can understand.
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read a really interesting study on the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but the same focus did ...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","Hey! I just rea

In [11]:
relevance_dict = prompt_chaining_dict(relevance_prompts_df, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

relevance_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='relevance')

summary_iteration_id: 1
n_choices: 1
**text1_prompt00
relevance_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate limit
relevance_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate lim

In [16]:
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


In [15]:
relevance_dict

,choice,article_title,system_role,model,prep step,summarization task,summary,text,preceding summary,relevance choice,relevance statement,relevance task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertr...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A study on long-term resistance training found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. The study suggests that attentional focus strategies affect hypertr...",1,"For athletes who want to optimize their muscle growth, these findings suggest that focusing on internal cues, such as the feeling of the muscle contracting, may be more effective for certain muscle groups. Incorporating this attentional focus strategy into their resistance training routine could...",Add 1-2 sentences to make this relevant for people who enjoy sports
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that different attentional focus strategies have varying effects on long-term muscular adaptations during resistance training. An internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but attentional focus strategy di...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A new study has found that different attentional focus strategies have varying effects on long-term muscular adaptations during resistance training. An internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but attentional focus strategy di...",1,"For athletes and fitness enthusiasts, this study provides valuable insights into the importance of attentional focus during resistance training. By understanding the impact of internal and external focus on muscle hypertrophy, athletes can tailor their training strategies to achieve their desire...",Add 1-2 sentences to make this relevant for people who enjoy sports
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read a really interesting study on the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but the same focus did ...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced seve

## 1.1
* Change column order
* Remove sleep print statements if no pause

In [20]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        print('relevance_iteration: ', relevance_iteration)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index, prompt in enumerate(simplify_prompts):
            # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            # print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_outputs=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "Added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    return results_dict


iteration_id = 1.1
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
relevance_prompts_df

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    # chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# simple_summaries_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[iteration_id]

# relevance_dict = prompt_chaining_dict(relevance_prompts_df, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (8, 8)


In [21]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 1.1
n_choices: 1
**text1_prompt00
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
simplify_iteration:  2
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 're

TypeError: process_chaining_results() missing 1 required positional argument: 'iteration_id'

In [22]:
simple_summaries_dict = dict()
simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

simple summaries dataframe shape: (16, 12)


,choice,article_title,system_role,model,text,prep step,summarization task,summary,original summary,simple summary,simple summary choice,simplify task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...","A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...","In this study, researchers looked at how different ways of thinking about your muscles during weightlifting affects how much your muscles grow over time. They found that focusing on your muscles from the inside (like thinking about how they feel or contract) is better for making your arm muscles...",1,Assume your audience has no science background.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study sug...","A new study has found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study sug...","In a recent study, researchers found that when people focus their attention on the muscles they are working on during resistance training, they are more likely to see muscle growth in their arms. However, this doesn't seem to be the case for leg muscles. This means that paying attention to the m...",1,Assume your audience has no science background.
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...","A recent study investigated the effe

In [24]:
relevance = prompt_chaining_dict(relevance_prompts_df, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )
relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
relevance_dict[iteration_id]

summary_iteration_id: 1.1
n_choices: 1
**text1_prompt00
relevance_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
relevance_iteration:  2
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given


,choice,article_title,system_role,model,text,prep step,summarization task,summary,preceding summary,relevance choice,relevance statement,relevance task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...","A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...",1,"For seniors who are interested in resistance training, this study suggests that focusing on internal cues, such as the sensation of the muscle contracting, may be more effective for building muscle in the arms. However, it is important to keep in mind that the study had limitations and may not a...",Add 1-2 sentences to make this relevant for seniors
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study sug...","A new study has found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study sug...",1,"For seniors who are interested in resistance training, this study suggests that focusing on the muscles being worked during exercise may be more beneficial for upper body muscle growth. However, attentional focus did not seem to affect lower extremity hypertrophy, so seniors may want to focus on...",Add 1-2 sentences to make this relevant for seniors
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus when the goal is to maximize hypertrophy of the elbow flexors. However, ...","A recent study investigate

# Iteration 2
Split up the relevance task into task and audience

In [26]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_outputs=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "Added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    return results_dict


iteration_id = 2
n_choices = 5
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
relevance_prompts_df

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
    # chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# simple_summaries = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (20, 9)


In [28]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"In a recent study, researchers investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,Exciting new research shows that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. This is the first study to investigate the effects of different attentional focus strategies on muscular adaptatio...
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research suggests that an internal focus of attention is superior to an external focus when maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found that attentional focus did not affect lower extremity hypertrophy, which may be due to the difficulty ..."
3,4,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"Attentional focus strategies have been found to have differential effects on long-term resistance training. A study showed that an internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, while attentional focus did not affect lower extremity..."
4,5,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance traini

## 2.1
Modify results processing to include save option

In [29]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2
n_choices = 5
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

# qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id)

simple_summaries_dict = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )
simple_summaries = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

summary_iteration_id: 2
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use terms a 12-year-old can understand. seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_05
	Done creating

[{1: {'response_01': {0: {'simple summary choice': 1,
     'simplify task': 'Use terms a 12-year-old can understand.',
     'audience': 'seniors',
     'full simplify task': 'Use terms a 12-year-old can understand. for seniors',
     'simple summary': 'Scientists did a study to see how people can make their muscles bigger by lifting weights. They found that thinking about the muscles you are using while lifting weights can make some muscles bigger, but not all of them. This is important for people who already exercise a lot, but they need to do more research to be sure.',
     'original summary': 'In a recent study, researchers investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. These findings suggest that cueing strategies c

In [41]:
simple_summaries_dict[iteration_id]

[{1: {'response_01': {0: {'simple summary choice': 1,
     'simplify task': 'Use terms a 12-year-old can understand.',
     'audience': 'seniors',
     'full simplify task': 'Use terms a 12-year-old can understand. for seniors',
     'simple summary': 'Scientists did a study to see how people can make their muscles bigger by lifting weights. They found that thinking about the muscles you are using while lifting weights can make some muscles bigger, but not all of them. This is important for people who already exercise a lot, but they need to do more research to be sure.',
     'original summary': 'In a recent study, researchers investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. These findings suggest that cueing strategies c

In [42]:
process_chaining_results(simple_summaries_dict[iteration_id], qna_dict, chatbot_dict, iteration_id)

simple summaries dataframe shape: (350, 285)


{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use terms a 12-year-old can understand.',
    'audience': 'seniors',
    'full simplify task': 'Use terms a 12-year-old can understand. for seniors',
    'simple summary': "Did you know that when people are mean to others because of their weight, it can make them do unhealthy things like eating too much or drinking too much alcohol? This is what a big study found out about lots of people in the United States. Even people who are not very overweight can be treated badly because of their weight. This can make them feel bad and hurt their health. So, it's important to be kind to everyone, no matter what they look like, to help keep everyone healthy.",
    'original summary': 'Hey, did you know that weight stigma is associated with poorer health behaviors? This was found in a two-stage research investigation conducted on a large sample of U.S. adults. The study found that weight stigma was significantly associated w

In [30]:
simple_summaries_dict[iteration_id]

[{1: {'response_01': {0: {'simple summary choice': 1,
     'simplify task': 'Use terms a 12-year-old can understand.',
     'audience': 'seniors',
     'full simplify task': 'Use terms a 12-year-old can understand. for seniors',
     'simple summary': 'Scientists did a study to see how people can make their muscles bigger by lifting weights. They found that thinking about the muscles you are using while lifting weights can make some muscles bigger, but not all of them. This is important for people who already exercise a lot, but they need to do more research to be sure.',
     'original summary': 'In a recent study, researchers investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but had no effect on lower extremity hypertrophy. These findings suggest that cueing strategies c

## 2.2

In [43]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2.2
n_choices = 5
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# simple_summaries = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (20, 9)


### qna_dict

In [44]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study shows that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus doesn't seem to affect lower extremity hypertrophy. The study also found marked differences in pe..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study shows that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy of elbow flexors during long-term resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. The study also suggests that differe..."
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research has found that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. The study also found marked differe..."
3,4,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not have an effect on lower extremity hypertrophy. The study also suggests that muscle size..."
4,5,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance trai

In [50]:
simple_summaries = prompt_chaining_dict(simplify_prompts, [''], simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id)
simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

summary_iteration_id: 2.2
n_choices: 1
**text1_prompt00
simplify_iteration:  2
Task: Use terms a 12-year-old can understand. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_05
	Done creating prom

AttributeError: 'list' object has no attribute 'keys'

In [55]:
print(len(simple_summaries[iteration_id]))
simple_summaries[iteration_id]

8


[{1: {'response_01': {0: {'simple summary choice': 1,
     'simplify task': 'Use terms a 12-year-old can understand.',
     'audience': 'seniors',
     'full simplify task': 'Use terms a 12-year-old can understand. for seniors',
     'simple summary': "A new study found that when you focus on the muscles you're using while lifting weights, your muscles grow bigger and stronger. But this only works for your arm muscles, not your leg muscles. The study also found that how you focus your attention affects how strong you are when you lift weights. This means that if you focus on your muscles while lifting weights, you can make them grow bigger and stronger.",
     'original summary': "New study shows that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus doesn't seem to affect lower extremity hypertrophy. The study also found marked differences in peak isometric s

## Edit processing step

In [63]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict

        print(results_dict)
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2.2
n_choices = 5
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

# qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
simple_summaries = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

{1: {'response_01': {0: {'simple summary choice': 1, 'simplify task': 'Use terms a 12-year-old can understand.', 'audience': 'seniors', 'full simplify task': 'Use terms a 12-year-old can understand. for seniors', 'simple summary': "A new study found that when you focus on the muscles you're using while lifting weights, your muscles grow bigger and stronger. But this only works for your arm muscles, not your leg muscles. The study also found that how you focus your attention affects how strong you are when you lift weights. This means that if you focus on your muscles while lifting weights, you can make them grow bigger and stronger.", 'original summary': "New study shows that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus doesn't seem to affect lower extremity hypertrophy. The study also found marked differences in peak isometric strength between internal a

AttributeError: 'list' object has no attribute 'keys'

## Iteration 2.3

In [64]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict

        print(results_dict)
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2.3
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# simple_summaries = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (12, 9)


### qna_dict

In [86]:
qna_dict[2.3].columns

Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')

In [65]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus, but has no effect on lower extremity hypertrophy. Internal focus may enhance the ""mind-muscle connection"" and lead to greate..."
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New findings from a study on long-term resistance training indicate that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not affect lower extremity hypertrophy. The study produced several novel and notable findings, includ..."
3,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey there! I just read an interesting study on the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but there wa..."
4,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differe

In [66]:
simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 2.3
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use terms a 12-year-old can understand. seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
simplify_iteration:  2
Task: Use terms a 12-year-old can understand. people who enjoy sports
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 c

In [69]:

simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

{1: {'response_01': {0: {'simple summary choice': 1, 'simplify task': 'Use terms a 12-year-old can understand.', 'audience': 'seniors', 'full simplify task': 'Use terms a 12-year-old can understand. for seniors', 'simple summary': "A new study found that when you focus on the muscles you're working on while lifting weights, it helps them grow bigger. This works better than just focusing on the weights or other things around you. The study only looked at people who were new to lifting weights, so we need more research to see if this works for people who have been lifting weights for a long time. Also, this only applies to the muscles in your arms, not your legs.", 'original summary': 'New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and external focus, an

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,...,53,54,55,56,57,58,59,60,61,62
0,1.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...,seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus, but has no effect on lower extremity hypertrophy. Internal focus may enhance the ""mind-muscle connection"" and lead to greate...",seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New findings from a study on long-term resistance training indicate that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not affect lower extremity hypertrophy. The study produced several novel and notable findings, includ...",seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...,people who enjoy sports,...,NaN,N

## 2.4

In [85]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict

        print(results_dict)
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                print('response_key:', response_key)
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2.4
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

# qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
chatbot_id = 2.3
simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id)
simple_summaries_dict[chatbot_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

{1: {'response_01': {0: {'simple summary choice': 1, 'simplify task': 'Use terms a 12-year-old can understand.', 'audience': 'seniors', 'full simplify task': 'Use terms a 12-year-old can understand. for seniors', 'simple summary': "A new study found that when you focus on the muscles you're working on while lifting weights, it helps them grow bigger. This works better than just focusing on the weights or other things around you. The study only looked at people who were new to lifting weights, so we need more research to see if this works for people who have been lifting weights for a long time. Also, this only applies to the muscles in your arms, not your legs.", 'original summary': 'New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and external focus, an

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,...,242,243,244,245,246,247,248,249,250,251
0,1.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...,seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus, but has no effect on lower extremity hypertrophy. Internal focus may enhance the ""mind-muscle connection"" and lead to greate...",seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New findings from a study on long-term resistance training indicate that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not affect lower extremity hypertrophy. The study produced several novel and notable findings, includ...",seniors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...,people who enjoy sports

In [81]:
simple_summaries_dict

{}

In [83]:
qna_dict[2.3]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,New study finds that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors during long-term resistance training. The study produced several notable findings including differences in changes in elbow flexor size between internal and externa...
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus, but has no effect on lower extremity hypertrophy. Internal focus may enhance the ""mind-muscle connection"" and lead to greate..."
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New findings from a study on long-term resistance training indicate that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not affect lower extremity hypertrophy. The study produced several novel and notable findings, includ..."
3,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey there! I just read an interesting study on the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but there wa..."
4,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differe

In [77]:
chatbot_dict

{1: {'text1_prompt00': <__main__.Chaining at 0x1f08525d610>,
  'text1_prompt01': <__main__.Chaining at 0x1f0a77409d0>,
  'text2_prompt00': <__main__.Chaining at 0x1f0a775cc40>,
  'text2_prompt01': <__main__.Chaining at 0x1f0a775cd00>},
 1.1: {'text1_prompt00': <__main__.Chaining at 0x1f0ab577d60>,
  'text1_prompt01': <__main__.Chaining at 0x1f0ab5ca5e0>,
  'text2_prompt00': <__main__.Chaining at 0x1f0a774feb0>,
  'text2_prompt01': <__main__.Chaining at 0x1f0ab5ca820>},
 2: {'text1_prompt00': <__main__.Chaining at 0x1f0ab61d760>,
  'text1_prompt01': <__main__.Chaining at 0x1f0ab607d60>,
  'text2_prompt00': <__main__.Chaining at 0x1f0ab5ca9d0>,
  'text2_prompt01': <__main__.Chaining at 0x1f0ab61acd0>},
 2.2: {'text1_prompt00': <__main__.Chaining at 0x1f0ac334a00>,
  'text1_prompt01': <__main__.Chaining at 0x1f0ac2c8820>,
  'text2_prompt00': <__main__.Chaining at 0x1f0ab607220>,
  'text2_prompt01': <__main__.Chaining at 0x1f0ac2c8970>},
 2.3: {'text1_prompt00': <__main__.Chaining at 0x1f0

## 2.5

In [7]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict

        print(results_dict)
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                print('response_key:', response_key)
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return results_dict


iteration_id = 2.5
n_choices = 3
qna_dict = dict()
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
no_audience = ['']

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# chatbot_id = 2.3
# simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[chatbot_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (12, 9)


### qna

In [8]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research suggests that attentional focus during resistance training can have differential effects on muscular adaptations. An internal focus was found to lead to superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle thickness in the quadriceps was una..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"""New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also highlights the importance of cueing s..."
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study shows that an internal focus of attention during resistance training can lead to superior hypertrophic increases in the elbow flexors compared to external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also suggests that differences in hypertrop..."
3,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey, did you know that a recent study found that using an internal focus of attention during long-term resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus? In fact, the difference in changes in elbow flexor size between internal and ..."
4,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differ

In [9]:
simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 2.5
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use terms a 12-year-old can understand. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
simplify_iteration:  2
Task: Assume your audience has no science background. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-t

### simple summaries dict

In [11]:

simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id, chatbot_id=iteration_id)
simple_summaries_dict[iteration_id]

{1: {'response_01': {0: {'simple summary choice': 1, 'simplify task': 'Use terms a 12-year-old can understand.', 'audience': '', 'full simplify task': 'Use terms a 12-year-old can understand. for ', 'simple summary': "A new study found that when people lift weights, thinking about the muscles they are using can help them grow bigger. But this only works for some muscles, like the ones in your arms, not your legs. The study was done on people who don't usually lift weights, so we need more research to see if it works for people who already exercise a lot.", 'original summary': 'New research suggests that attentional focus during resistance training can have differential effects on muscular adaptations. An internal focus was found to lead to superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle thickness in the quadriceps was unaffected by attentional focus strategy. This indicates that a "mind-muscle connection" may enhance muscle growth in speci

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,full simplify task,original summary,simple summary,simple summary choice,simplify task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research suggests that attentional focus during resistance training can have differential effects on muscular adaptations. An internal focus was found to lead to superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle thickness in the quadriceps was una...",,Use terms a 12-year-old can understand. for,"New research suggests that attentional focus during resistance training can have differential effects on muscular adaptations. An internal focus was found to lead to superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle thickness in the quadriceps was una...","A new study found that when people lift weights, thinking about the muscles they are using can help them grow bigger. But this only works for some muscles, like the ones in your arms, not your legs. The study was done on people who don't usually lift weights, so we need more research to see if i...",1,Use terms a 12-year-old can understand.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"""New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also highlights the importance of cueing s...",,Use terms a 12-year-old can understand. for,"""New study finds that an internal focus of attention during resistance training leads to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also highlights the importance of cueing s...","A new study found that when you focus on the muscles you are using while lifting weights, your arm muscles get bigger. But it doesn't make a difference for your leg muscles. The study also found that how you are told to lift weights can help you get bigger muscles.",1,Use terms a 12-year-old can understand.
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New stu

### simple summaries df columns

In [13]:
[column for column in simple_summaries_dict[iteration_id].columns]

['choice',
 'article_title',
 'system_role',
 'model',
 'text',
 'prep step',
 'summarization task',
 'full summarization task',
 'summary',
 'audience',
 'full simplify task',
 'original summary',
 'simple summary',
 'simple summary choice',
 'simplify task']

In [27]:
chaining_dict[2.5]['text2_prompt01'].simple_summary_dict

{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use terms a 12-year-old can understand.',
    'audience': '',
    'full simplify task': 'Use terms a 12-year-old can understand. for ',
    'simple summary': 'Hey there! Did you know that when people are made to feel bad about their weight, it can make them do things that are not good for their health, like eating too much or drinking too much alcohol? This was found in a study of many people in the US. Even if someone is not very heavy, they can still be affected by weight stigma. However, the study did not find a link between weight stigma and how much people exercise. The study also suggests that feeling bad about your weight can make you stressed, which can also affect your health. Overall, the study shows that making people feel bad about their weight can stop them from being healthy, and we should try to be kinder to people of all sizes to help everyone be healthier.',
    'original summary': 'Hey! Did you

## 2.6

In [29]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} for {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict

        print(results_dict)
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                print('response_key:', response_key)
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    ).drop(columns=['original summary'])
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


iteration_id = 2.6
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
no_audience = ['']

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# chatbot_id = 2.3
# simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
# simple_summaries_dict[iteration_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (12, 9)


In [30]:
simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )


summary_iteration_id: 2.6
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use terms a 12-year-old can understand. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
simplify_iteration:  2
Task: Assume your audience has no science background. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-t

In [31]:
chatbot_id = 2.3
simple_summaries_dict = process_chaining_results(simple_summaries_dict, qna_dict, chatbot_dict, iteration_id)
simple_summaries_dict[iteration_id]

{1: {'response_01': {0: {'simple summary choice': 1, 'simplify task': 'Use terms a 12-year-old can understand.', 'audience': '', 'full simplify task': 'Use terms a 12-year-old can understand. for ', 'simple summary': "A new study found that when people focus on their muscles while lifting weights, their arm muscles get bigger. But this didn't work as well for leg muscles because it's harder to think about them while exercising. The study also found that people who focused on their muscles were stronger when they held a weight still. But we need more research to see if this works for people who already have experience lifting weights.", 'original summary': 'New study finds that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty of establishing a "mind-muscle connection" in th

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,full simplify task,simple summary,simple summary choice,simplify task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty of establ...",,Use terms a 12-year-old can understand. for,"A new study found that when people focus on their muscles while lifting weights, their arm muscles get bigger. But this didn't work as well for leg muscles because it's harder to think about them while exercising. The study also found that people who focused on their muscles were stronger when t...",1,Use terms a 12-year-old can understand.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A new study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but did not affec...",,Use terms a 12-year-old can understand. for,"Scientists did a study to see how people can focus their attention while lifting weights. They found that thinking about the muscles you are using (internal focus) helps your arm muscles get bigger, but doesn't make your leg muscles bigger. They also found that thinking about the movement of the...",1,Use terms a 12-year-old can understand.
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that focusing internally on the mind-muscle connection during resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found that di...",,Use terms a 12-year-old can understand. for,"Scientists did a study on how to make muscles bigger and stronger by lifting weights. They found that if you think about your muscles while you lift weights, your arm muscles can get bigger and stronger. But thinking about your muscles didn't make y

### 2.61

In [38]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    # new_results_columns = new_results.columns.tolist()
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )#.drop(columns=['original summary'])
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            # 'original summary',
            'simple summary'
        ]
    else:
        columns = new_results.columns.tolist()
        # columns= [

        # ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


iteration_id = 2.61
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
no_audience = ['']

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

# qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
chatbot_id = 2.6
simple_summaries_dict = process_chaining_results(
    simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
    chatbot_id=chatbot_id
    )
simple_summaries_dict[chatbot_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

New results shape: (23, 6)
simple summaries dataframe shape: (23, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience,simplify task,full simplify task,simple summary
0,Differential effects of attentional focus strategies during long-term resistance training,1,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty of establ...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand. for,"A new study found that when people focus on their muscles while lifting weights, their arm muscles get bigger. But this didn't work as well for leg muscles because it's harder to think about them while exercising. The study also found that people who focused on their muscles were stronger when t..."
1,Differential effects of attentional focus strategies during long-term resistance training,2,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"A new study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but did not affec...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand. for,"Scientists did a study to see how people can focus their attention while lifting weights. They found that thinking about the muscles you are using (internal focus) helps your arm muscles get bigger, but doesn't make your leg muscles bigger. They also found that thinking about the movement of the..."
2,Differential effects of attentional focus strategies during long-term resistance training,3,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that focusing internally on the mind-muscle connection during resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found that di...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand. for,"Scientists did a study on how to make muscles bigger and stronger by lifting weights. They found that if you think about your muscles while you lift weights, your arm muscles can get bigger and stronger. But

In [42]:
relevance_prompts = task_part3
iteration_id = 2.6
relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )
relevance_dict = process_chaining_results(
    relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance', chatbot_id=chatbot_id,
    save_df=True, save_chatbot=True
    )

KeyError: 2.61

In [ ]:

relevance_dict[iteration_id]

## 2.7

In [43]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    # new_results_columns = new_results.columns.tolist()
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )#.drop(columns=['original summary'])
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            # 'original summary',
            'simple summary'
        ]
    else:
        columns = new_results.columns.tolist()
        # columns= [

        # ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


iteration_id = 2.7
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
no_audience = ['']

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# chatbot_id = 2.6
# simple_summaries_dict = process_chaining_results(
#     simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
#     chatbot_id=chatbot_id
#     )
# simple_summaries_dict[chatbot_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
# relevance_dict[iteration_id]

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (12, 9)


### qna

In [44]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that an internal focus of attention during resistance training can lead to greater muscle growth in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found marked differences in isometric elbo..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"This study found that an internal focus of attention during resistance training resulted in greater hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. These findings suggest that cueing strategies can affe..."
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy in the elbow flexors during resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy for untrained individuals. The study also f..."
3,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey there! I just read an interesting study on the effects of different attentional focus strategies during long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but muscl..."
4,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differe

In [45]:
simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )


summary_iteration_id: 2.7
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: Use terms a 12-year-old can understand. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
simplify_iteration:  2
Task: Assume your audience has no science background. 
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-t

### simple summaries dict

In [46]:
chatbot_id = iteration_id
simple_summaries_dict = process_chaining_results(
    simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
    chatbot_id=chatbot_id
    )
simple_summaries_dict[chatbot_id]

New results shape: (24, 6)
simple summaries dataframe shape: (24, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'simple summary choice', 'audience', 'simplify task', 'full simplify task', 'simple summary']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,simple summary choice,audience,simplify task,full simplify task,simple summary
0,Differential effects of attentional focus strategies during long-term resistance training,1,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that an internal focus of attention during resistance training can lead to greater muscle growth in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found marked differences in isometric elbo...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand.,"A new study found that when people focus on their muscles while lifting weights for their arms, they can make their muscles grow bigger. But this doesn't work for their legs. The study also found that people who focused on their muscles had stronger arm muscles than those who didn't. This means ..."
1,Differential effects of attentional focus strategies during long-term resistance training,2,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"This study found that an internal focus of attention during resistance training resulted in greater hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. These findings suggest that cueing strategies can affe...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand.,"This study looked at how people think about their muscles while lifting weights. They found that when people think about their muscles working inside their body, their arm muscles get bigger. But thinking about the weights or the movement doesn't make a difference. This means that how you think ..."
2,Differential effects of attentional focus strategies during long-term resistance training,3,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy in the elbow flexors during resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy for untrained individuals. The study also f...",1,,Use terms a 12-year-old can understand.,Use terms a 12-year-old can understand.,"A new study found that when you're lifting weights, it's better to focus on the muscles you're using inside your body instead of what's happening outside. This helps your muscles grow bigger in your arms. But for people

In [47]:
relevance_prompts = task_part3
relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )


summary_iteration_id: 2.7
n_choices: 1
**text1_prompt00
relevance_iteration:  1
Task: Add 1-2 sentences to make this relevant for seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
relevance_iteration:  2
Task: Add 1-2 sentences to make this relevant for people who enjoy sports
summaries_keys: 
	 ['response_01', 'response_02', 'response_03']
		...Preparing to add relevance to re

### relevance_dict

In [48]:
relevance_dict = process_chaining_results(relevance_dict, qna_dict, chatbot_dict, iteration_id, results_type='relevance')
relevance_dict[iteration_id]

New results shape: (24, 6)
added relevance dataframe shape: (24, 15)
['choice', 'article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'audience', 'full relevance task', 'preceding summary', 'relevance choice', 'relevance statement', 'relevance task']


,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,full relevance task,preceding summary,relevance choice,relevance statement,relevance task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that an internal focus of attention during resistance training can lead to greater muscle growth in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found marked differences in isometric elbo...",seniors,Add 1-2 sentences to make this relevant for seniors,"New research shows that an internal focus of attention during resistance training can lead to greater muscle growth in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found marked differences in isometric elbo...",1,"For seniors who engage in resistance training, adopting an internal focus of attention during upper body exercises may lead to greater muscle growth in the arms. However, attentional focus did not affect lower body muscle growth. It is important to consult with a healthcare professional before s...",Add 1-2 sentences to make this relevant for
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"This study found that an internal focus of attention during resistance training resulted in greater hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. These findings suggest that cueing strategies can affe...",seniors,Add 1-2 sentences to make this relevant for seniors,"This study found that an internal focus of attention during resistance training resulted in greater hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. These findings suggest that cueing strategies can affe...",1,"For seniors who engage in resistance training, it is important to consider the type of cues they use during their workouts. This study suggests that focusing internally on the muscle being worked may lead to greater muscle growth in the arms, but may not have the same effect on leg muscles. Seni...",Add 1-2 sentences to make this relevant for
2,3,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key p

#### columns

In [50]:
relevance_dict[iteration_id].columns.tolist()

['choice',
 'article_title',
 'system_role',
 'model',
 'text',
 'prep step',
 'summarization task',
 'full summarization task',
 'summary',
 'audience',
 'full relevance task',
 'preceding summary',
 'relevance choice',
 'relevance statement',
 'relevance task']

## 2.71 columns are in desired order

In [51]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_chaining_results(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    # new_results_columns = new_results.columns.tolist()
    print('New results shape:', new_results.shape)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        # columns = new_results.columns.tolist()
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            # 'preceding summary',
            'relevance statement'
        ]
    new_results = new_results[columns]
    print(f'{"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                results_dict[iteration_id], description='prompt_chain_simple_summaries',
                csv_path=csv_path, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            save_output(
                chatbot_dict[iteration_id], description='chaining_bot',
                csv_path=None, pickle_path=pickle_path)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


iteration_id = 2.71
n_choices = 3
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()
# relevance_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2
no_audience = ['']

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

# qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id)

# simple_summaries = prompt_chaining_dict(simplify_prompts, no_audience, simple_summaries_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
# chatbot_id = 2.6
# simple_summaries_dict = process_chaining_results(
#     simple_summaries_dict, qna_dict, chatbot_dict, iteration_id,
#     chatbot_id=chatbot_id
#     )
# simple_summaries_dict[chatbot_id]

# relevance_prompts = task_part3
# relevance = prompt_chaining_dict(relevance_prompts, audience, relevance_dict, 
#     chaining_dict[summary_iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )
chatbot_id = 2.7
relevance_dict = process_chaining_results(
    relevance_dict, qna_dict, chatbot_dict, iteration_id, 
    results_type='relevance', chatbot_id=chatbot_id
    )
relevance_dict[chatbot_id]

New results shape: (24, 6)
added relevance dataframe shape: (24, 14)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
0,Differential effects of attentional focus strategies during long-term resistance training,1,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New research shows that an internal focus of attention during resistance training can lead to greater muscle growth in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. The study also found marked differences in isometric elbo...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors who engage in resistance training, adopting an internal focus of attention during upper body exercises may lead to greater muscle growth in the arms. However, attentional focus did not affect lower body muscle growth. It is important to consult with a healthcare professional before s..."
1,Differential effects of attentional focus strategies during long-term resistance training,2,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"This study found that an internal focus of attention during resistance training resulted in greater hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. These findings suggest that cueing strategies can affe...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors who engage in resistance training, it is important to consider the type of cues they use during their workouts. This study suggests that focusing internally on the muscle being worked may lead to greater muscle growth in the arms, but may not have the same effect on leg muscles. Seni..."
2,Differential effects of attentional focus strategies during long-term resistance training,3,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post,"New study finds that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy in the elbow flexors during resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy for untrained individuals. The study also f...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for seniors,"For seniors who are interested in resistance training, this study suggests that focusing on internal cues, such as the feeling of the muscle contrac

In [52]:
relevance_dict[chatbot_id].sort_values(by=['audience']).head(2)

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
11,Differential effects of attentional focus strategies during long-term resistance training,3,You are an expert at science communication.,gpt-3.5-turbo,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey! So, I just read a study that looked at the effects of different attentional focus strategies on long-term muscular adaptations. They found that an internal focus (focusing on the muscle being worked) produced superior hypertrophic increases in the elbow flexors compared to an external focus...",1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for people who enjoy sports,"For athletes looking to maximize their muscle growth and strength gains, this study highlights the importance of focusing on the muscle being worked during training. By developing a strong mind-muscle connection, athletes may be able to achieve greater hypertrophic adaptations and improve their ..."
21,Weight stigma and health behaviors: evidence from the Eating in America Study,1,You are an expert at science communication.,gpt-3.5-turbo,"Weight stigma and health behaviors: evidence from the Eating in America Study\n\nDISCUSSION \n\nThe present study employed a two-stage research investigation to examine the relationship between weight stigma and several health behaviors in a large sample of U.S. adults. As predicted, weight stig...",Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler,"Hey, did you know about a recent study that shows weight stigma is linked to several unhealthy behaviors in adults? The study found that weight stigma is associated with disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. Surprisingly, even individuals with...",1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for people who enjoy sports,"This study is particularly relevant for individuals who enjoy sports as it highlights the negative impact of weight stigma on overall health, including athletic performance. It emphasizes the need for a weight-inclusive approach to health promotion, which can benefit athletes of all body types a..."


# *End of Page*